**Initial Setup**

In [ ]:
# Import python packages
import lime
import lime.lime_tabular
import pandas as pd
import numpy as np

# We can also use Snowpark for our analyses!
#from snowflake.snowpark.context import get_active_session
from snowflake.snowpark import Session
#session = get_active_session()
connection_parameters = {
   "account": "...", # REPLACE WITH YOURS
   "user": "...", # REPLACE WITH YOURS
   "authenticator":'...', 
   "role": "...",  # REPLACE WITH YOURS
   "warehouse": "...",  # REPLACE WITH YOURS
   "database": "...",  # REPLACE WITH YOURS
   "schema": "...",  # REPLACE WITH YOURS
   }

new_session = Session.builder.configs(connection_parameters).create()


**Create Training View** -  https://quickstarts.snowflake.com/guide/lead_scoring_with_ml_powered_classification/index.html?index=..%2F..index#0

In [26]:
cmd=r"""create or replace view customer_training as select age_band, household_income, marital_status,
 household_size, case when total_order_value<10 then 'BRONZE' 
 when total_order_value<=25 and total_order_value>10 then 'SILVER'
 else 'GOLD' END as segment from customers where join_date<'2024-02-11'::date;
 """

new_session.sql(cmd).collect()


[Row(status='View CUSTOMER_TRAINING successfully created.')]

**Encode Textual Labels, as LIME does not work on Textual Values**

In [27]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

df_training_data = new_session.table('customer_training').to_pandas()

feat = ['AGE_BAND', 'HOUSEHOLD_INCOME_le', 'MARITAL_STATUS_le', 'HOUSEHOLD_SIZE']
# label encoding textual data
df_training_data['HOUSEHOLD_INCOME_le'] = le.fit_transform(df_training_data['HOUSEHOLD_INCOME'])
df_training_data['MARITAL_STATUS_le'] = le.fit_transform(df_training_data['MARITAL_STATUS']) 
df_training_data = df_training_data.drop(columns=["HOUSEHOLD_INCOME","MARITAL_STATUS"])

#save dataframe
sp_df = new_session.create_dataframe(df_training_data)
sp_df.write.mode("overwrite").save_as_table("customer_training_encoded")

**View new Table with Encoded Values**

In [28]:
cmd = r"""SELECT * FROM customer_training_encoded;"""
new_session.sql(cmd).collect()

[Row(AGE_BAND=50, HOUSEHOLD_SIZE=3.0, SEGMENT='GOLD', HOUSEHOLD_INCOME_le=5, MARITAL_STATUS_le=1),
 Row(AGE_BAND=75, HOUSEHOLD_SIZE=4.0, SEGMENT='GOLD', HOUSEHOLD_INCOME_le=1, MARITAL_STATUS_le=1),
 Row(AGE_BAND=30, HOUSEHOLD_SIZE=1.0, SEGMENT='BRONZE', HOUSEHOLD_INCOME_le=2, MARITAL_STATUS_le=0),
 Row(AGE_BAND=40, HOUSEHOLD_SIZE=2.0, SEGMENT='SILVER', HOUSEHOLD_INCOME_le=3, MARITAL_STATUS_le=2),
 Row(AGE_BAND=30, HOUSEHOLD_SIZE=1.0, SEGMENT='SILVER', HOUSEHOLD_INCOME_le=1, MARITAL_STATUS_le=1),
 Row(AGE_BAND=70, HOUSEHOLD_SIZE=2.0, SEGMENT='BRONZE', HOUSEHOLD_INCOME_le=5, MARITAL_STATUS_le=0),
 Row(AGE_BAND=30, HOUSEHOLD_SIZE=5.0, SEGMENT='BRONZE', HOUSEHOLD_INCOME_le=2, MARITAL_STATUS_le=1),
 Row(AGE_BAND=45, HOUSEHOLD_SIZE=2.0, SEGMENT='SILVER', HOUSEHOLD_INCOME_le=1, MARITAL_STATUS_le=1),
 Row(AGE_BAND=35, HOUSEHOLD_SIZE=2.0, SEGMENT='GOLD', HOUSEHOLD_INCOME_le=1, MARITAL_STATUS_le=1),
 Row(AGE_BAND=60, HOUSEHOLD_SIZE=6.0, SEGMENT='SILVER', HOUSEHOLD_INCOME_le=4, MARITAL_STATUS_le=

**Train ML Powered Classification Model on Encoded Training View**

In [29]:
cmd = r"""
-- create the classification model
CREATE OR REPLACE SNOWFLAKE.ML.CLASSIFICATION customer_classification_model(
    INPUT_DATA => SYSTEM$REFERENCE('table', 'customer_training_encoded'),
    TARGET_COLNAME => 'segment'
);
"""
new_session.sql(cmd).collect()

[Row(status='Instance CUSTOMER_CLASSIFICATION_MODEL successfully created.')]

**Create Temporary Table for Predictions**

In [30]:
cmd = r"""
-- run prediction and save results
CREATE OR REPLACE TEMPORARY TABLE customer_predictions_prep AS
SELECT age_band, household_income, marital_status, household_size
from customers;
"""
new_session.sql(cmd).collect()

[Row(status='Table CUSTOMER_PREDICTIONS_PREP successfully created.')]

**Encode Textual Values in Prediction Table and Save**

In [31]:
df_predictions_data = new_session.table('customer_predictions_prep').to_pandas()

feat = ['AGE_BAND', 'HOUSEHOLD_INCOME_le', 'MARITAL_STATUS_le', 'HOUSEHOLD_SIZE']
# label encoding textual data
df_predictions_data['HOUSEHOLD_INCOME_le'] = le.fit_transform(df_predictions_data['HOUSEHOLD_INCOME'])
df_predictions_data['MARITAL_STATUS_le'] = le.fit_transform(df_predictions_data['MARITAL_STATUS']) 
df_predictions_data = df_predictions_data.drop(columns=["HOUSEHOLD_INCOME","MARITAL_STATUS"])

sp_df = new_session.create_dataframe(df_predictions_data)
#print(sp_df.to_pandas().to_string())
sp_df.write.mode("overwrite").save_as_table("customer_predictions_encoded")

**Create a new Table from Temp Table with Numerical (Encoded) Values**

In [34]:
cmd = r"""
-- run prediction and save results
CREATE OR REPLACE TABLE customer_predictions AS
SELECT AGE_BAND, "HOUSEHOLD_INCOME_le", "MARITAL_STATUS_le", HOUSEHOLD_SIZE,
customer_classification_model!PREDICT(INPUT_DATA => object_construct(*)) as predictions,
PARSE_JSON(PREDICTIONS)['class']::String as predicted_segment
from customer_predictions_encoded;
"""

new_session.sql(cmd).collect()

[Row(status='Table CUSTOMER_PREDICTIONS successfully created.')]

**Load Table into Snowpark Dataframe** 

In [35]:
df_prediction_data = new_session.table('customer_predictions').to_pandas()

**Set Up LIME TABULAR Explainer for Classification**

In [36]:
feat = ['AGE_BAND', 'HOUSEHOLD_INCOME_le', 'MARITAL_STATUS_le', 'HOUSEHOLD_SIZE']

explainer = lime.lime_tabular.LimeTabularExplainer(df_prediction_data[feat].astype(int).values,
mode='classification',training_labels=np.array(df_prediction_data['PREDICTED_SEGMENT']),feature_names=feat)

**Ask the Explainer to explain the first row of predictions**

In [37]:
#ML Powered Classification does not have a predict_proba function, so we need to create a custom function
#to generate the probabilities for each class and return them in a 2D numpy array
def prob(data):
    return_array = []
    for i in data:     
        cmd ="""CREATE OR REPLACE TEMPORARY TABLE run_classification AS 
        SELECT customer_classification_model!PREDICT(INPUT_DATA => object_construct('AGE_BAND', ?, 'HOUSEHOLD_INCOME_le', ?, 'MARITAL_STATUS_le', ?, 'HOUSEHOLD_SIZE', ?)) as predictions"""

        age = float(i[0])
        hh_income = float(i[1])
        mstatus = float(i[2])
        hh_size = float(i[3])
        
        new_session.sql(cmd, params=[age, hh_income, mstatus, hh_size]).collect()

        cmd ="""
            CREATE OR REPLACE TEMPORARY TABLE predict_proba_classificationone AS
            SELECT
                predictions:probability:BRONZE::FLOAT AS bronze_proba,
                predictions:probability:SILVER::FLOAT AS silver_proba,
                predictions:probability:GOLD::FLOAT AS gold_proba,
            FROM run_classification,
            LATERAL FLATTEN(input => predictions);
        """
        new_session.sql(cmd).collect()
        
        proba_array = new_session.table('predict_proba_classificationone').to_pandas().iloc[0].tolist()
        #print(proba_array)
        return_array.append(proba_array)
        #print(return_array)
    return np.array(return_array)

# asking for explanation for LIME model, first row
i = 1
exp = explainer.explain_instance(df_prediction_data.loc[i,feat].astype(int).values, prob, num_features=4, num_samples=7)

**What is the First Row of Predictions?**

In [38]:
i=1
print(df_prediction_data.loc[i,feat].astype(int).values)

[40  5  1  3]


**Show Explanation For First Row in a List**

In [39]:
exp.as_list()

[('4.00 < HOUSEHOLD_INCOME_le <= 5.00', -0.12144896721903205),
 ('2.00 < HOUSEHOLD_SIZE <= 3.00', -0.08836519795153007),
 ('MARITAL_STATUS_le <= 1.00', -0.06891969813957886),
 ('AGE_BAND <= 40.00', 0.029510121082555126)]